GOALS: 

      CNN-LSTM/Spatial-Temporial LSTM https://towardsdatascience.com/spatial-temporal-convlstm-for-crash-prediction-411909ed2cfa
      Figure out how to break data into depth, lat and lon


#Getting Data

In [2]:
import requests 
file_url = "https://iridl.ldeo.columbia.edu/SOURCES/.CARTON-GIESE/.SODA/.v2p2p4/ssh/data.nc"
	
r = requests.get(file_url, stream = True) 

with open("./data.nc", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

import os
size = os.path.getsize('./data.nc') 
print('Size of file is', size, 'bytes')

Size of file is 1573874020 bytes


### SSH

In [1]:
import xarray as xr
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ds_ssh = xr.open_dataset("./data.nc" , decode_times=False)
#ds_ssh = xr.open_dataset("http://iridl.ldeo.columbia.edu/SOURCES/.CARTON-GIESE/.SODA/.v2p2p4/ssh/dods" , decode_times=False)

In [4]:
def fix_calendar(ds, timevar='time'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

ds_ssh = fix_calendar(ds_ssh)
ds_ssh = xr.decode_cf(ds_ssh)
ds_ssh

<xarray.Dataset>
Dimensions:  (lat: 330, lon: 720, time: 1656)
Coordinates:
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    ssh      (time, lat, lon) float32 ...

In [5]:
ds_ssh

<xarray.Dataset>
Dimensions:  (lat: 330, lon: 720, time: 1656)
Coordinates:
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    ssh      (time, lat, lon) float32 ...

In [ ]:
ds_ssh = ds_ssh.where(((ds_ssh.lon > 20 ) & (ds_ssh.lon < 80 ) & (ds_ssh.lat > 10) & (ds_ssh.lat <20)), drop=True)

In [ ]:
ds_ssh

<xarray.Dataset>
Dimensions:  (lat: 20, lon: 120, time: 1656)
Coordinates:
  * lon      (lon) float32 20.25 20.75 21.25 21.75 ... 78.25 78.75 79.25 79.75
  * lat      (lat) float32 10.25 10.75 11.25 11.75 ... 18.25 18.75 19.25 19.75
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    ssh      (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

In [ ]:
ds_ssh['ssh']

<xarray.DataArray 'ssh' (time: 1656, lat: 20, lon: 120)>
array([[[       nan,        nan,        nan, ...,        nan,
         0.47368455, 0.4705949 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.47322145],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.45891967],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
         0.49529132, 0.49132225],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.4916404 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.47177067],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
         0.552477  , 0.5522664 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.5573779 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.54433674],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 20.25 20.75 21.25 21.75 ... 78.25 78.75 79.25 79.75
  * lat      (lat) float32 10.25 10.75 11.25 11.75 ... 18.25 18.75 19.25 19.75
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Attributes:
    pointwidth:     1.0
    units:          meters
    standard_name:  sea_surface_height
    long_name:      SEA SURFACE HEIGHT

In [ ]:
np.array(ds_ssh['ssh']).shape

(1656, 20, 120)

In [ ]:
np.array(ds_ssh['ssh'][1650][0])

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan, -0.01835824,  0.02075754,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,  0.02848718,  0.0399644 ,  0.09675951,  0.13464183,
        0.15300493,  0.20257978,  0.28198448,  0.38239264,  0.47

### Salt

In [17]:


file = 'salt.mon.ltm.nc'
data = nc.Dataset(file)

In [18]:
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    creation_date: Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux: 
Note that the net surface heat flux are the total surface heat flux 
from the NCEP reanalysis 2 plus the relaxation terms.
    time_comment: The internal time stamp indicates the FIRST day of the averaging period.
    Conventions: COARDS
    grib_file: godas.M.199101-12.grb
    html_REFERENCES: http://www.cpc.ncep.noaa.gov/products/GODAS/background.shtml
    html_BACKGROUND: http://www.cpc.ncep.noaa.gov/products/GODAS/background.shtml
    html_GODAS: www.cpc.ncep.noaa.gov/products/GODAS
    comment: NOTE:  THESE ARE THE BIAS CORRECTED GODAS FILES.
    title: GODAS: Global Ocean Data Assimilation System
    References: https://www.psl.noaa.gov/data/gridded/data.godas.html
    dataset_title: NCEP Global Ocean Data Assimilation System (GODAS)
    history: Created 2022/01/27 by doMonthLTMannualFileInputs
    not_missing_threshold_pe

In [20]:
ds_salt = xr.open_dataset('salt.mon.ltm.nc' , decode_times=False)
ds_salt

<xarray.Dataset>
Dimensions:             (lat: 418, level: 40, lon: 360, nbnds: 2, time: 12)
Coordinates:
  * lat                 (lat) float32 -74.5 -74.17 -73.83 ... 63.83 64.17 64.5
  * lon                 (lon) float32 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
  * level               (level) float32 5.0 15.0 25.0 ... 3.972e+03 4.478e+03
  * time                (time) float64 -6.571e+05 -6.57e+05 ... -6.567e+05
Dimensions without coordinates: nbnds
Data variables:
    climatology_bounds  (time, nbnds) float64 ...
    salt                (time, level, lat, lon) float32 ...
    valid_yr_count      (time, level, lat, lon) float32 ...
Attributes: (12/14)
    creation_date:                  Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:                    \nNote that the net surface heat flux are...
    time_comment:                   The internal time stamp indicates the FIR...
    Conventions:                    COARDS
    grib_file:                      godas.M.199101-12.grb
    html_REFERENCES:                http://www.cpc.ncep.noaa.gov/products/GOD...
    ...                             ...
    comment:                        NOTE:  THESE ARE THE BIAS CORRECTED GODAS...
    title:                          GODAS: Global Ocean Data Assimilation System
    References:                     https://www.psl.noaa.gov/data/gridded/dat...
    dataset_title:                  NCEP Global Ocean Data Assimilation Syste...
    history:                        Created 2022/01/27 by doMonthLTMannualFil...
    not_missing_threshold_percent:  minimum 3% values input to have non-missi...

In [23]:
ds_salt['salt']

<xarray.DataArray 'salt' (time: 12, level: 40, lat: 418, lon: 360)>
[72230400 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 -74.5 -74.17 -73.83 -73.5 ... 63.5 63.83 64.17 64.5
  * lon      (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * level    (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * time     (time) float64 -6.571e+05 -6.57e+05 ... -6.568e+05 -6.567e+05
Attributes: (12/15)
    dataset:                  NCEP GODAS
    var_desc:                 salinity
    level_desc:               Multiple Levels
    statistic:                Long Term Mean
    parent_stat:              Monthly Mean
    sub_center:               Environmental Modeling Center
    ...                       ...
    level_indicator:          160
    gds_grid_type:            0
    parameter_table_version:  2
    parameter_number:         88
    valid_range:              [0.  0.1]
    actual_range:             [0.01442202 0.04435723]

In [24]:
ds_salt.keys()

KeysView(<xarray.Dataset>
Dimensions:             (lat: 418, level: 40, lon: 360, nbnds: 2, time: 12)
Coordinates:
  * lat                 (lat) float32 -74.5 -74.17 -73.83 ... 63.83 64.17 64.5
  * lon                 (lon) float32 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
  * level               (level) float32 5.0 15.0 25.0 ... 3.972e+03 4.478e+03
  * time                (time) float64 -6.571e+05 -6.57e+05 ... -6.567e+05
Dimensions without coordinates: nbnds
Data variables:
    climatology_bounds  (time, nbnds) float64 6.976e+04 8.035e+04 ... 8.069e+04
    salt                (time, level, lat, lon) float32 ...
    valid_yr_count      (time, level, lat, lon) float32 ...
Attributes: (12/14)
    creation_date:                  Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:                    \nNote that the net surface heat flux are...
    time_comment:                   The internal time stamp indicates the FIR...
    Conventions:                    COARDS
    grib_file:                   

In [25]:
np.array(ds_salt['salt'])

array([[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [0.0352031 , 0.03519201, 0.03517757, ..., 0.03521337,
          0.03521175, 0.03520961],
         [0.03519893, 0.03518916, 0.03517563, ..., 0.03520788,
          0.03520666, 0.03520453],
         [0.03519466, 0.03518621, 0.03517329, ..., 0.03520279,
          0.03520158, 0.03519964]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [0.0352029 , 0.03519191, 0.03517747, ..., 0.03521358,
          0.03521175, 0.0

In [33]:
np.array(ds_salt['salt']).shape

(12, 40, 418, 360)

In [29]:
#ds_salt = xr.open_dataarray(url, chunks={'S': 1, 'L': 1})

ds_salt_sub = ds_salt.where(((ds_salt.lon>2.5 )&(ds_salt.lat>=-73.83) & (ds_salt.level>=5.0) ), drop=True)

In [30]:
ds_salt_sub

<xarray.Dataset>
Dimensions:             (lat: 415, level: 40, lon: 357, nbnds: 2, time: 12)
Coordinates:
  * lat                 (lat) float32 -73.5 -73.17 -72.83 ... 63.83 64.17 64.5
  * lon                 (lon) float32 3.5 4.5 5.5 6.5 ... 357.5 358.5 359.5
  * level               (level) float32 5.0 15.0 25.0 ... 3.972e+03 4.478e+03
  * time                (time) float64 -6.571e+05 -6.57e+05 ... -6.567e+05
Dimensions without coordinates: nbnds
Data variables:
    climatology_bounds  (time, nbnds, lon, lat, level) float64 6.976e+04 ... ...
    salt                (time, level, lat, lon) float32 nan nan nan ... nan nan
    valid_yr_count      (time, level, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes: (12/14)
    creation_date:                  Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:                    \nNote that the net surface heat flux are...
    time_comment:                   The internal time stamp indicates the FIR...
    Conventions:                    COARDS
    grib_file:                      godas.M.199101-12.grb
    html_REFERENCES:                http://www.cpc.ncep.noaa.gov/products/GOD...
    ...                             ...
    comment:                        NOTE:  THESE ARE THE BIAS CORRECTED GODAS...
    title:                          GODAS: Global Ocean Data Assimilation System
    References:                     https://www.psl.noaa.gov/data/gridded/dat...
    dataset_title:                  NCEP Global Ocean Data Assimilation Syste...
    history:                        Created 2022/01/27 by doMonthLTMannualFil...
    not_missing_threshold_percent:  minimum 3% values input to have non-missi...

In [31]:

ds_salt_sub['salt']

<xarray.DataArray 'salt' (time: 12, level: 40, lat: 415, lon: 357)>
array([[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [0.03516607, 0.03515813, 0.0351445 , ..., 0.03521337,
          0.03521175, 0.03520961],
         [0.03516373, 0.03515447, 0.03514196, ..., 0.03520788,
          0.03520666, 0.03520453],
         [0.03516109, 0.03515041, 0.03513972, ..., 0.03520279,
          0.03520158, 0.03519964]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
...
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 -73.5 -73.17 -72.83 -72.5 ... 63.5 63.83 64.17 64.5
  * lon      (lon) float32 3.5 4.5 5.5 6.5 7.5 ... 355.5 356.5 357.5 358.5 359.5
  * level    (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * time     (time) float64 -6.571e+05 -6.57e+05 ... -6.568e+05 -6.567e+05
Attributes: (12/15)
    dataset:                  NCEP GODAS
    var_desc:                 salinity
    level_desc:               Multiple Levels
    statistic:                Long Term Mean
    parent_stat:              Monthly Mean
    sub_center:               Environmental Modeling Center
    ...                       ...
    level_indicator:          160
    gds_grid_type:            0
    parameter_table_version:  2
    parameter_number:         88
    valid_range:              [0.  0.1]
    actual_range:             [0.01442202 0.04435723]

In [32]:
np.array(ds_salt_sub['salt']).shape

(12, 40, 415, 357)

### Temperatures

In [37]:


file2 = 'pottmp.mon.ltm.nc'
datatemp = nc.Dataset(file2)

In [38]:
datatemp

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    creation_date: Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux: 
Note that the net surface heat flux are the total surface heat flux 
from the NCEP reanalysis 2 plus the relaxation terms.
    time_comment: The internal time stamp indicates the FIRST day of the averaging period.
    Conventions: COARDS
    grib_file: godas.M.199101-12.grb
    html_REFERENCES: http://www.cpc.ncep.noaa.gov/products/GODAS/background.shtml
    html_BACKGROUND: http://www.cpc.ncep.noaa.gov/products/GODAS/background.shtml
    html_GODAS: www.cpc.ncep.noaa.gov/products/GODAS
    comment: NOTE:  THESE ARE THE BIAS CORRECTED GODAS FILES.
    title: GODAS: Global Ocean Data Assimilation System
    References: https://www.psl.noaa.gov/data/gridded/data.godas.html
    dataset_title: NCEP Global Ocean Data Assimilation System (GODAS)
    history: Created 2022/01/27 by doMonthLTMannualFileInputs
    not_missing_threshold_pe

In [39]:
ds_temp = xr.open_dataset('pottmp.mon.ltm.nc' , decode_times=False)
ds_temp

<xarray.Dataset>
Dimensions:             (lat: 418, level: 40, lon: 360, nbnds: 2, time: 12)
Coordinates:
  * lat                 (lat) float32 -74.5 -74.17 -73.83 ... 63.83 64.17 64.5
  * lon                 (lon) float32 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
  * level               (level) float32 5.0 15.0 25.0 ... 3.972e+03 4.478e+03
  * time                (time) float64 -6.571e+05 -6.57e+05 ... -6.567e+05
Dimensions without coordinates: nbnds
Data variables:
    climatology_bounds  (time, nbnds) float64 ...
    pottmp              (time, level, lat, lon) float32 ...
    valid_yr_count      (time, level, lat, lon) float32 ...
Attributes: (12/14)
    creation_date:                  Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:                    \nNote that the net surface heat flux are...
    time_comment:                   The internal time stamp indicates the FIR...
    Conventions:                    COARDS
    grib_file:                      godas.M.199101-12.grb
    html_REFERENCES:                http://www.cpc.ncep.noaa.gov/products/GOD...
    ...                             ...
    comment:                        NOTE:  THESE ARE THE BIAS CORRECTED GODAS...
    title:                          GODAS: Global Ocean Data Assimilation System
    References:                     https://www.psl.noaa.gov/data/gridded/dat...
    dataset_title:                  NCEP Global Ocean Data Assimilation Syste...
    history:                        Created 2022/01/27 by doMonthLTMannualFil...
    not_missing_threshold_percent:  minimum 3% values input to have non-missi...

In [40]:
ds_temp.keys()

KeysView(<xarray.Dataset>
Dimensions:             (lat: 418, level: 40, lon: 360, nbnds: 2, time: 12)
Coordinates:
  * lat                 (lat) float32 -74.5 -74.17 -73.83 ... 63.83 64.17 64.5
  * lon                 (lon) float32 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
  * level               (level) float32 5.0 15.0 25.0 ... 3.972e+03 4.478e+03
  * time                (time) float64 -6.571e+05 -6.57e+05 ... -6.567e+05
Dimensions without coordinates: nbnds
Data variables:
    climatology_bounds  (time, nbnds) float64 6.976e+04 8.035e+04 ... 8.069e+04
    pottmp              (time, level, lat, lon) float32 ...
    valid_yr_count      (time, level, lat, lon) float32 ...
Attributes: (12/14)
    creation_date:                  Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:                    \nNote that the net surface heat flux are...
    time_comment:                   The internal time stamp indicates the FIR...
    Conventions:                    COARDS
    grib_file:                   

In [49]:
np.array(ds_temp['pottmp'])

array([[[[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [280.14847, 280.32437, 280.4759 , ..., 279.69638, 279.80844,
          279.9817 ],
         [280.05774, 280.23282, 280.3878 , ..., 279.59216, 279.71536,
          279.88672],
         [279.96664, 280.1414 , 280.29965, ..., 279.48785, 279.62244,
          279.79187]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [280.15488, 280.3299 , 280.48138, ..., 279.7034 , 279.81528,
          279.98834],
         [280.06393, 280.23856, 280.39343, ..., 27

In [50]:
np.array(ds_temp['pottmp']).shape

(12, 40, 418, 360)

In [44]:
ds_temp_sub = ds_temp.where(((ds_temp.lon>2.5 )&(ds_temp.lat>=-73.83) & (ds_temp.level>=5.0) ), drop=True)

In [45]:
ds_temp_sub

<xarray.Dataset>
Dimensions:             (lat: 415, level: 40, lon: 357, nbnds: 2, time: 12)
Coordinates:
  * lat                 (lat) float32 -73.5 -73.17 -72.83 ... 63.83 64.17 64.5
  * lon                 (lon) float32 3.5 4.5 5.5 6.5 ... 357.5 358.5 359.5
  * level               (level) float32 5.0 15.0 25.0 ... 3.972e+03 4.478e+03
  * time                (time) float64 -6.571e+05 -6.57e+05 ... -6.567e+05
Dimensions without coordinates: nbnds
Data variables:
    climatology_bounds  (time, nbnds, lon, lat, level) float64 6.976e+04 ... ...
    pottmp              (time, level, lat, lon) float32 nan nan nan ... nan nan
    valid_yr_count      (time, level, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
Attributes: (12/14)
    creation_date:                  Sat Dec 16 20:00:00 MDT 2006
    sfcHeatFlux:                    \nNote that the net surface heat flux are...
    time_comment:                   The internal time stamp indicates the FIR...
    Conventions:                    COARDS
    grib_file:                      godas.M.199101-12.grb
    html_REFERENCES:                http://www.cpc.ncep.noaa.gov/products/GOD...
    ...                             ...
    comment:                        NOTE:  THESE ARE THE BIAS CORRECTED GODAS...
    title:                          GODAS: Global Ocean Data Assimilation System
    References:                     https://www.psl.noaa.gov/data/gridded/dat...
    dataset_title:                  NCEP Global Ocean Data Assimilation Syste...
    history:                        Created 2022/01/27 by doMonthLTMannualFil...
    not_missing_threshold_percent:  minimum 3% values input to have non-missi...

In [47]:
ds_temp_sub['pottmp']

<xarray.DataArray 'pottmp' (time: 12, level: 40, lat: 415, lon: 357)>
array([[[[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [280.6168 , 280.7211 , 280.75015, ..., 279.69638, 279.80844,
          279.9817 ],
         [280.52844, 280.6329 , 280.66174, ..., 279.59216, 279.71536,
          279.88672],
         [280.43967, 280.54492, 280.57318, ..., 279.48785, 279.62244,
          279.79187]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
...
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 -73.5 -73.17 -72.83 -72.5 ... 63.5 63.83 64.17 64.5
  * lon      (lon) float32 3.5 4.5 5.5 6.5 7.5 ... 355.5 356.5 357.5 358.5 359.5
  * level    (level) float32 5.0 15.0 25.0 ... 3.483e+03 3.972e+03 4.478e+03
  * time     (time) float64 -6.571e+05 -6.57e+05 ... -6.568e+05 -6.567e+05
Attributes: (12/15)
    dataset:                  NCEP GODAS
    var_desc:                 potential temperature
    level_desc:               Multiple Levels
    statistic:                Long Term Mean
    parent_stat:              Monthly Mean
    sub_center:               Environmental Modeling Center
    ...                       ...
    level_indicator:          160
    gds_grid_type:            0
    parameter_table_version:  2
    parameter_number:         13
    valid_range:              [260. 310.]
    actual_range:             [270.72812 305.5059 ]

In [48]:
np.array(ds_temp_sub['pottmp']).shape

(12, 40, 415, 357)

# Data Manpulation

In [ ]:
!pip install netCDF4

In [ ]:
ds = nc.Dataset('data.nc')

#Data Visulazation

In [ ]:
!pip install cartopy

In [ ]:
import cartopy.crs as ccrs

In [ ]:
days = np.arange(0,365)
# creating meshgrid from our list of spatial coordinates
lon, lat = np.meshgrid(longs,lats)
# generate a frame for each day
for i in days:
    # set the axes' spatial projection, styling, and title
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_title('Precipitation for day '+ str(i+1))
    
    # generate the contour map
    plt.contourf(longs, lats, tp[i,], 60,
    transform=ccrs.PlateCarree())
    
    # save the figure and give a pause for visualization
    plt.savefig(f'./output/{i+1}.jpg')
    plt.pause(1)
plt.show()

In [ ]:
import PIL
image_frames = []
days = np.arange(20,80) # sub-sample day number 20 to 80
for k in days:
    new_frame = PIL.Image.open(f'./output/{k}.jpg') 
    image_frames.append(new_frame)
# save as GIF
image_frames[0].save('tp_timelapse.gif',
                     format='GIF',
                     append_images = image_frames[1: ],
                     save_all = True, 
                     duration = 100,
                     loop = 0)